In [1]:
library('tidyverse')
library('tidylog')
library('httr')
library('jsonlite')
# Aplicar funcoes em paralelo
library('future.apply')


# Estrutura de pastas
pasta_dados        <- "../../yellow_dados"
pasta_orig_vs_mod  <- sprintf('%s/10_rotas_originais_vs_modeladas', pasta_dados)
pasta_orig_way_ids <- sprintf("%s/01_osm_way_ids_rotas_modeladas", pasta_orig_vs_mod)
dir.create(pasta_orig_way_ids, recursive = TRUE, showWarnings = FALSE)

── Attaching packages ─────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.1     ✔ dplyr   1.0.5
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘tidylog’


The following objects are masked from ‘package:dplyr’:

    add_count, add_tally, anti_join, count, distinct, distinct_all,
    distinct_at, distinct_if, filter, filter_all, filter_at, filter_if,
    full_join, group_by, group_by_all, group_by_at, group_by_if,
    inner_join, left_join, mutate, mutate_all, mutate_at, mutate_if,
    relocate, rename, rename_all, rename_at, rename_if, rename_with,
    right_join, sample_frac, sample_n, select, select_all, select_at,
    select_if, semi_join, slice, 

In [2]:
# Checando: Jupyter suporta multicore?
future::supportsMulticore()

[1] TRUE

In [3]:
# https://stackoverflow.com/questions/40536067/how-to-adjust-future-global-maxsize
# For 850MB: 850*1024^2 = 891289600
# For +1.3GB: 1500*1024^2 = 1572864000
# options(future.globals.maxSize = 891289600)

In [11]:
# ------------------------------------------------------------------------------
# Routing a partir de dois pontos com rotas alternativas (até 3 por par OD)
# ------------------------------------------------------------------------------

# Faz query de routing no GraphHopper e retorna resultados principais em dataframe,
# com rotas até 3 alternativas por par OD - aceita um dataframe de uma linha como
# entrada
gh_route <- function(tripid, out_file, route_options) {
  # url <- 'http://localhost:8989/route/?point=-23.5314933121698%2C-46.634354542765&point=-23.5390199310058%2C-46.6376369484305&profile=bike&instructions=false&calc_points=true&details=average_speed'
  # url <- ods_vgs %>% slice(1) %>% select(url) %>% pull()
  # Estação Vila Madalena: -23.546258,-46.690898
  # IME: -23.559007,-46.73208
  # CCSP: -23.571498,-46.639806
  
  # df_line$url <- paste('http://localhost:8989/route/?point=', 
  # '-23.541504', '%2C', '-46.685779', '&point=', 
  # '-23.571498', '%2C', '-46.639806', route_options,
  # sep = '')
  
  df_line <- ods_vgs %>% filter(trip_id == tripid)
  # df_line <- tmp_df
  
  # Fazer a GET de roteamento no Grahphopper
  # print(df_line$url)
  gh_response <- GET(df_line$url)
  
  # Mensagem tem que ser "Success: (200) OK"
  if (http_status(gh_response)$message == 'Success: (200) OK') {
    
    # Resposta da query, já colapsada e transformada em dataframe
    # Remover aviso de 'No encoding supplied: defaulting to UTF-8' na linha fromJSON()
    suppressMessages(
      response_text <- 
        # Ignorar aviso 'argument is not an atomic vector; coercing'
        suppressWarnings(str_c(content(gh_response, 'text'), collapse = ", ")) %>% 
        # Concatenar toda a string de resultados
        str_c("[", ., "]") %>% 
        # Transformar em dataframe
        fromJSON() %>% 
        as.data.frame()
    )
    
    # Nos interessa a coluna de 'paths', como um novo dataframe
    paths <- response_text$paths %>% as.data.frame()
    
    # Puxar osm_way_ids dos resultados 
    osm_ways <- paths$details$osm_way_id[1] %>% as.data.frame()
      
    if (nrow(osm_ways) > 0) {
      osm_ways <- osm_ways %>% mutate(index = 1:nrow(.)) %>% select(index, osm_way_id = X3)
      
      # Gravar resultados
      osm_way_out <- sprintf('%s/%s.csv', pasta_orig_way_ids, df_line$trip_id)
      write_delim(osm_ways, osm_way_out, delim = ';')
      
    } else {
      # Se não há osm_way_ids, é porque os pontos estão muito próximos uns dos
      # outros, mesmo que seja um osm_way_id diferente entre a origem e o 
      # destino. A distância e a velocidade calculadas vão ser zero também - 
      # pular este registro, que vai ser vazio
      return(sprintf('Pulando: %s não tem osm_way_ids (provavelmente tem distância = 0)', df_line$trip_id))
    }
     
    
    
    # Isolar colunas de interesse
    paths <- 
      paths %>% 
      # Calcular tempo em segundos e velocidade média
      mutate(time = time / 1000,
             speed = distance / time * 3.6) %>% 
      # Descartar colunas extras - a coluna poly é o shape da rota traçada
      select(distance, weight, time, speed, poly = points)
    
    # Testar polyline:
    # https://valhalla.github.io/demos/polyline/?unescape=true&polyline6=false#%0A
    
    # Adicionar colunas de informação vindas do dataframe original
    paths <- 
      paths %>% 
      mutate(trip_id   = df_line$trip_id,
             qgis_id.x = df_line$qgis_id.x,
             qgis_id.y = df_line$qgis_id.y,
             .before = 'distance') %>% 
      mutate(lon.x     = df_line$lon.x,
             lat.x     = df_line$lat.x,
             lon.y     = df_line$lon.y,
             lat.y     = df_line$lat.y,
             .after = 'poly')
    
  } else {
    
    # Se a query no GraphHopper não deu resultados, guardar como dataframe vazio
    paths <- data.frame(trip_id   = df_line$trip_id,
                        qgis_id.x = df_line$qgis_id.x,
                        qgis_id.y = df_line$qgis_id.y,
                        distance  = NA,
                        weight    = NA,
                        time      = NA,
                        speed     = NA,
                        poly      = NA,
                        lon.x     = df_line$lon.x,
                        lat.x     = df_line$lat.x,
                        lon.y     = df_line$lon.y,
                        lat.y     = df_line$lat.y
    )
    
  }
  
  # Guardar resultados temporários
  if (file.exists(out_file)) {
    write_delim(paths, out_file, delim = ';', append = TRUE)
  } else {
    write_delim(paths, out_file, delim = ';', append = FALSE)
  }
  
}

In [10]:
# ------------------------------------------------------------------------------
# Routing a partir de dois pontos
# ------------------------------------------------------------------------------

# Abrir origens e destinos das rotas iniciais da Yellow - são consideradas aqui
# (pelos scripts anteriores) as rotas que (a) tiveram algum trecho considerado
# no modelo; (b) que não foram divididas em trechos menores; e (c) em que o
# trecho único considerado é o inicial (possui trip_id com _00)
ods_vgs <- sprintf('%s/03_rotas_originais_infraciclo_detour_carac_viagens.csv', pasta_orig_vs_mod)
ods_vgs <- read_delim(ods_vgs, delim = ';', col_types = 'cccdddddccccdddddddd')
# Remover viagens com origem e destino no mesmo qgis_id
ods_vgs <- ods_vgs %>% filter(qgis_id.x != qgis_id.y)
ods_vgs <- ods_vgs %>% select(trip_id, qgis_id.x, qgis_id.y, lon.x, lat.x, lon.y, lat.y)
# head(ods_vgs)

# Criar coluna com URL para GET no GraphHopper
route_options <- '&profile=bike&instructions=false&calc_points=true&details=osm_way_id'

ods_vgs <- 
  ods_vgs %>% 
  mutate(url = paste('http://localhost:8989/route/?point=', 
                     lat.x, '%2C', lon.x, '&point=', 
                     lat.y, '%2C', lon.y, route_options,
                     sep = ''))


# Arquivo de saída
out_file <- sprintf('%s/04_ttmatrix_viagens_modeladas_a_partir_das_originais.csv', pasta_orig_vs_mod)

# Se arquivo de saída existe, desconsiderar viagens que já foram roteadas
if (file.exists(out_file)) {
  this <- read_delim(out_file, delim = ';', col_types = cols(.default = "c"))
  this <- this %>% select(trip_id) %>% distinct()
  
  # Remover viagens que já foram roteadas
  ods_vgs <- ods_vgs %>% filter(!trip_id %in% this$trip_id)
  rm(this)
  
}

nrow(ods_vgs)
head(ods_vgs)

filter: no rows removed

select: dropped 13 variables (via_comum, infra_ciclo, ciclo_expressa, ciclo_comum, ciclofaixa, …)

mutate: new variable 'url' (character) with 129,026 unique values and 0% NA



[1] 129755

trip_id,qgis_id.x,qgis_id.y,lon.x,lat.x,lon.y,lat.y,url
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
000135_00,271157,104168,-46.71660,-23.56869,-46.72109,-23.57161,http://localhost:8989/route/?point=-23.56869%2C-46.716596&point=-23.571614%2C-46.721093&profile=bike&instructions=false&calc_points=true&details=osm_way_id
000138_00,161929,161977,-46.68779,-23.57601,-46.69842,-23.57327,http://localhost:8989/route/?point=-23.576007%2C-46.687787&point=-23.573271%2C-46.698423&profile=bike&instructions=false&calc_points=true&details=osm_way_id
000139_00,031336,143030,-46.69835,-23.57297,-46.68893,-23.56802,http://localhost:8989/route/?point=-23.572974%2C-46.698347&point=-23.568022%2C-46.688926&profile=bike&instructions=false&calc_points=true&details=osm_way_id
000140_00,143030,255789,-46.68880,-23.56799,-46.69571,-23.57371,http://localhost:8989/route/?point=-23.567988%2C-46.688804&point=-23.573713%2C-46.695715&profile=bike&instructions=false&calc_points=true&details=osm_way_id
000141_00,255788,264853,-46.69577,-23.57374,-46.69206,-23.57702,http://localhost:8989/route/?point=-23.573744%2C-46.695772&point=-23.577017%2C-46.692057&profile=bike&instructions=false&calc_points=true&details=osm_way_id
000142_00,264853,031336,-46.69205,-23.57703,-46.69840,-23.57305,http://localhost:8989/route/?point=-23.577026%2C-46.692052&point=-23.573045%2C-46.698396&profile=bike&instructions=false&calc_points=true&details=osm_way_id


In [14]:
# No terminal, iniciar GraphHopper com .pbf da rede 2019 e config.yml sem alterações de LTS
# (base) livre@laika:/media/livre/HDINT1TB/Base_GtsRegionais$ clear && cd /home/livre/Desktop/Base_GtsRegionais/GitLab/yellow_src/graphhopper/ && rm -rf graph-cache/ && java -Ddw.graphhopper.datareader.file=/home/livre/Desktop/Base_GtsRegionais/GitLab/yellow_dados/07_graphhopper/03_PBFs_SP_rede_2019/20220216_sao_paulo_edited_20230521_A_infraciclo_atual.osm.pbf -jar graphhopper/web/target/graphhopper-web-*.jar server graphhopper/config-example.yml


# Criar ttmatrix a partir do GrahHopper
detach("package:tidylog")
# lapply(hex_com_vizinhos$url, gh_route, n = n_secao)

# Rodar função para todos os arquivos- multi thread (Jupyter)
(start = Sys.time())
future::plan(future::multicore)
invisible(future.apply::future_lapply(X   = ods_vgs$trip_id, 
                                      FUN = gh_route,
                                      out_file = out_file,
                                      route_options = route_options, 
                                      future.seed = TRUE))
Sys.time()
Sys.time() - start

[1] "2024-04-30 09:31:28 -03"

[1] "2024-04-30 09:58:02 -03"

Time difference of 26.56759 mins